# Generate result (in .ds file) based on input

# 1. Setup environment

In [4]:
import pandas as pd
import numpy as np
import json
import joblib
import tensorflow
from tensorflow.keras.preprocessing.sequence import pad_sequences

model_addr = './models/model_rnn_gru.h5'

# 2. Model Loading
Load the model and scalers

In [5]:
model = tensorflow.keras.models.load_model(model_addr)

# 3. Lyric Input
Make sure the lyric is been preprocessed in GPT

In [ ]:
# Input sequence
input_sequence = "AP n ei f a g e n a j i f u y u a p u AP n ei f a g e n a l e y u d ao en AP"

In [ ]:
# Load the encoding dictionary from the ph_token_to_int.json file
with open('ph_token_to_int.json', 'r') as file:
    ph_token_to_int = json.load(file)
    
# Split the input sequence into individual phonemes
input_phonemes = input_sequence.split()

# Convert the phonemes to their corresponding integers using the encoding dictionary
encoded_sequence = [ph_token_to_int[phoneme] for phoneme in input_phonemes]

# Convert the sequence to a numpy array and pad it to the right length
ph_seq_encoded = np.array([encoded_sequence]) 

user_input = pad_sequences(ph_seq_encoded, maxlen=283, padding='post', truncating='post', dtype='float64')


# 2. Inference
Generate result

In [ ]:
# Prediction
y_note_pred, y_dur_pred = model.predict(user_input)
print(y_note_pred.shape, y_note_pred.shape)

# 3. Decoding
Decode the prediction result

In [ ]:
# Resize the result
input_length = len(encoded_sequence)
y_note_pred = y_note_pred[:, :input_length, :]
y_dur_pred = y_dur_pred[:, :input_length, :]

# print(y_note_pred.shape, y_dur_pred.shape)

# Load the saved scaler
note_scaler = joblib.load('note_scaler.pkl')

y_note_pred_flattened = y_note_pred.flatten()

# Inverse transform the predictions
y_note_pred_original = note_scaler.inverse_transform(y_note_pred_flattened.reshape(-1, 1))

# Reshape back to the original shape if required
y_note_pred_original = y_note_pred_original.reshape(y_note_pred.shape)
y_note_pred_original = np.round(y_note_pred_original).astype(int)

f0_timestep = 0.005
f0_seq_constant = 250.1

total_time = np.sum(y_dur_pred)
print(total_time)
f0_size = int(total_time / f0_timestep)
# print(f0_size)

f0_seq = [f0_seq_constant] * f0_size

# 4. Output .ds file

In [ ]:
y_note_pred_flattened = y_note_pred_original[0]
y_note_pred_output = [int(i) for i in y_note_pred_flattened]

y_dur_pred_flattened = y_dur_pred[0]
y_dur_pred_output = [float(i) for i in y_dur_pred_flattened]

# Load the token-to-int mappings from the JSON files
def load_mapping(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        token_to_int = json.load(f)
    # Invert the dictionary to create an int-to-token mapping
    int_to_token = {v: k for k, v in token_to_int.items()}
    return int_to_token

# Decoding function using the mappings
def decode_predictions(prediction_integers, mapping):
    return [mapping.get(i, 'Unknown') for i in prediction_integers]

note_int_to_token = load_mapping('note_token_to_int.json')

decoded_note_seq = decode_predictions(y_note_pred_output, note_int_to_token)

# Print or return the decoded sequences
# print("Decoded Phonetic Sequence:", input_phonemes)
# print("Decoded Note Sequence:", decoded_note_seq)

ph_dur = ' '.join(map(str, y_dur_pred_output))
note_seq = ' '.join(map(str, decoded_note_seq))
f0_seq = ' '.join(map(str, f0_seq))

file = {
    'ph_seq': input_sequence,
    'ph_dur': ph_dur,
    'note_seq': note_seq,
    'f0_seq': f0_seq,
    'f0_timestep': f0_timestep
}

with open('rnn_gru_output.ds', 'w') as json_file:
    json.dump(file, json_file, indent=4)